In [20]:
import logging
import apache_beam as beam

from apache_beam.options.pipeline_options import PipelineOptions


In [21]:
# logging
root = logging.getLogger()
root.setLevel(logging.INFO)


In [22]:
# create a pipeline, read data from csv
pipeline_options = beam.Pipeline(options=PipelineOptions())

# PCollection format is = id, customer-type, product-type, price
transactions = (pipeline_options
                | "Read Transaction CSV"
                >> beam.io.ReadFromText("gs://spark-dataset-1/datasets/sales/sales_transactions.csv")
                # | "Raw transactions"
                # >> beam.CombineGlobally(beam.combiners.CountCombineFn())
                # | "Raw transactions count"
                # >> beam.ParDo(lambda c: logging.info("Total lines %s", str(c)))
                )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:oauth2client.client:Timeout attempting to reach GCE metadata service.
Connecting anonymously.
INFO:oauth2client.client:Timeout attempting to reach GCE metadata service.
Connecting anonymously.


In [23]:
# extract product-type and price
class ExtractProductTypePrice(beam.DoFn):

    def process(self, element, *args, **kwargs):
        line = element.split(',')
        return [(line[2], float(line[3]))]


# PCollection format now = product-type, price
productType_price = (transactions
                     | "Extracting product type and price"
                     >> beam.ParDo(ExtractProductTypePrice())
                     | "Product Type and Price"
                     >> beam.CombineGlobally(beam.combiners.CountCombineFn())
                     | "Product Type and Price count"
                     >> beam.ParDo(lambda c: logging.info("Total lines %s", str(c)))            
                     )


In [24]:
# run
result = pipeline_options.run()

result.wait_until_finish()

INFO:root:==================== <function annotate_downstream_side_inputs at 0x11921a9b0> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x11921aed8> ====================
INFO:root:==================== <function lift_combiners at 0x11921d320> ====================
INFO:root:==================== <function expand_gbk at 0x11921a938> ====================
INFO:root:==================== <function sink_flattens at 0x11921a7d0> ====================
INFO:root:==================== <function greedily_fuse at 0x11921af50> ====================
INFO:root:==================== <function impulse_to_input at 0x11921d398> ====================
INFO:root:==================== <function inject_timer_pcollections at 0x11921acf8> ====================
INFO:root:==================== <function sort_stages at 0x11921a8c0> ====================
INFO:root:Running ((ref_AppliedPTransform_Read Transaction CSV/Read_3)+((ref_AppliedPTransform_Extracting product type and price_

'DONE'